# LAB | Abstractive Question Answering

Abstractive question-answering focuses on the generation of multi-sentence answers to open-ended questions. It usually works by searching massive document stores for relevant information and then using this information to synthetically generate answers. This notebook demonstrates how Pinecone helps you build an abstractive question-answering system. We need three main components:

- A vector index to store and run semantic search
- A retriever model for embedding context passages
- A generator model to generate answers

# Install Dependencies

In [28]:
# !pip install -qU datasets pinecone-client==3.1.0 sentence-transformers torch

# Load and Prepare Dataset

Our source data will be taken from the Wiki Snippets dataset, which contains over 17 million passages from Wikipedia. But, since indexing the entire dataset may take some time, we will only utilize 50,000 passages in this demo that include "History" in the "section title" column. If you want, you may utilize the complete dataset. Pinecone vector database can effortlessly manage millions of documents for you.

In [27]:
from huggingface_hub import hf_hub_download

In [81]:
import re
import random
import pyarrow.parquet as pq
from datasets import load_dataset

## load the dataset from huggingface in streaming mode and shuffle it
# wiki_data = load_dataset(
#     'vblagoje/wikipedia_snippets_streamed',
#     split='train',
#     streaming=True
# ).shuffle(seed=960)


PARQUET_PATH = "wiki40b-train.parquet"

def buffered_shuffle(iterator, buffer_size=5000, seed=960):
    rng = random.Random(seed)
    buf = []
    for x in iterator:
        buf.append(x)
        if len(buf) >= buffer_size:
            rng.shuffle(buf)
            while buf:
                yield buf.pop()
    rng.shuffle(buf)
    while buf:
        yield buf.pop()

def iter_wiki40b_passages_with_titles(parquet_path, window=120, overlap=30):
    pf = pq.ParquetFile(parquet_path)
    step = max(1, window - overlap)

    for rg in range(pf.num_row_groups):
        batch = pf.read_row_group(rg).to_pydict()

        for raw_text, wid in zip(batch["text"], batch["wikidata_id"]):
            text = raw_text.replace("_NEWLINE_", "\n")
            lines = text.split("\n")

            # ---- Extract article title ----
            article_title = ""
            for i, line in enumerate(lines):
                if line == "_START_ARTICLE_" and i + 1 < len(lines):
                    article_title = lines[i + 1].strip()
                    break

            # ---- Extract section starts ----
            section_starts = []
            current_section = ""
            for i, line in enumerate(lines):
                if line == "_START_SECTION_" and i + 1 < len(lines):
                    current_section = lines[i + 1].strip()
                    section_starts.append((i, current_section))

            # ---- Build word map ----
            word_map = []
            for i, line in enumerate(lines):
                if line.startswith("_START_"):
                    continue
                for w in line.split():
                    word_map.append((i, w))

            if not word_map:
                continue

            # ---- Helper: find section for a given line ----
            def section_for_line(li):
                s = ""
                for start_i, title in section_starts:
                    if start_i <= li:
                        s = title
                    else:
                        break
                return s

            # ---- Chunking ----
            for chunk_id, start in enumerate(range(0, len(word_map), step)):
                chunk = word_map[start:start + window]
                if not chunk:
                    continue

                start_line = chunk[0][0]
                passage_text = " ".join(w for _, w in chunk).strip()

                yield {
                    "wiki_id": wid,
                    "article_title": article_title,
                    "section_title": section_for_line(start_line),
                    "chunk_id": chunk_id,
                    "passage_text": passage_text,
                }
# streaming + shuffle equivalent
wiki_data = buffered_shuffle(
    iter_wiki40b_passages_with_titles(PARQUET_PATH, window=120, overlap=30),
    buffer_size=5000,
    seed=960
)

doc = next(iter(wiki_data))
print(doc.keys())
print(doc["section_title"])
print(doc["passage_text"][:200])


dict_keys(['wiki_id', 'article_title', 'section_title', 'chunk_id', 'passage_text'])
Namesake
References to the 'Berliner Theater am Nollendorfplatz' in the 1930s mentioning Erwin Piscator, Berthold Brecht, Gustav von Wangenheim, Hans Meyer-Hanno and others, probably refer to the Neues Schausp


We are loading the dataset in the streaming mode so that we don't have to wait for the whole dataset to download (which is over 9GB). Instead, we iteratively download records one at a time.

In [82]:
item = next(iter(wiki_data))
item.keys(), item["passage_text"][:200]

(dict_keys(['wiki_id', 'article_title', 'section_title', 'chunk_id', 'passage_text']),
 "properly, though some onlookers noted Edward had a little difficulty placing the wedding ring on Sophie's finger. In keeping with tradition, the wedding ring was crafted from Welsh gold from the Princ")

In [83]:
# show the contents of a single document in the dataset
next(iter(wiki_data))

{'wiki_id': 'Q24521524',
 'article_title': 'The Jordan Museum',
 'section_title': '',
 'chunk_id': 0,
 'passage_text': "The Jordan Museum Background The Jordan Archaeological Museum was established in 1951 hosting Jordan's most important archaeological findings. However, the old site became too small and the idea of developing a new modern museum emerged in 2005. A joint committee headed by Queen Rania, became responsible for developing the idea of a new modern museum by international standards. Construction started in 2009 and the museum was officially opened in 2014, spanning over 10,000 square meters. Location The museum is located in the Ras Al-Ein area near downtown Amman, adjacent to the Greater Amman Municipality headquarters. The Museum is only a street away from major archaeological sites in Amman such as the Roman theater, Nymphaeum, Amman Citadel and The Hashemite Plaza."}

In [84]:
# filter only documents with History as section_title - Replace None with your code
history = (x for x in wiki_data if x.get("section_title")=='History')
next(iter(history))

{'wiki_id': 'Q24090072',
 'article_title': 'Surface Hill Uniting Church',
 'section_title': 'History',
 'chunk_id': 7,
 'passage_text': "to mark the construction of the new church. A stone-laying ceremony was held on 30 January 1890. Eight large foundation stones were laid by prominent members of the congregation, a representative of the local preachers and one of the Sunday school scholars. An earthenware jar was placed under the first stone that was laid and it contained a copy of that day's Gympie Times, the previous day's Gympie Miner, The Weekly Advocate, The Christian Witness, the circuit plan, a programme of the proposed proceedings of the ceremony, the society ticket for the last quarter (a Methodist membership ticket), a list of the Trustees and officers of the church, a list of the names of the teachers and officers of the"}

Let's iterate through the dataset and apply our filter to select the 50,000 historical passages. We will extract `article_title`, `section_title` and `passage_text` from each document.

In [85]:
from tqdm.auto import tqdm  # progress bar

total_doc_count = 50000

counter = 0
docs = []
# iterate through the dataset and apply our filter
for d in tqdm(history, total=total_doc_count):
    # extract the fields we need - article, section, and passage
    # print(counter)
    if counter >= total:
        break
    docs.append({'article_title':d['article_title'],'section_title':d['section_title'],'passage_text':d['passage_text']})
    # increase the counter on every iteration
    counter += 1
docs


  0%|          | 0/50000 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

[{'article_title': 'Terai',
  'section_title': 'History',
  'passage_text': "appointed government officers in the eastern Terai districts of Parsa, Bara, Rautahat, Mahottari, Saptari and Morang to levy taxes, collect revenues, and capture elephants and rhinos. The far-western and mid-western regions of the Nepal Terai called 'Naya Muluk' (new country) lay on the northern periphery of the Awadh dynasty. After Nepal lost the Anglo–Nepalese War in 1816, the British annexed these regions in the Terai when the Sugauli Treaty was ratified. But as reward for Nepal's military aid in the Indian Rebellion of 1857, they returned some of this region in 1860, namely today's districts Kanchanpur, Kailali, Banke and Bardiya. Dacoit gangs retreated to the Terai jungles, and the area was considered lawless and primitive by the British, who sought"},
 {'article_title': 'WJMX-FM',
  'section_title': 'History',
  'passage_text': 'by 1987 before it could be acted upon. In early 1988, WJMX-FM purchased WPDZ

In [86]:
import pandas as pd

# create a pandas dataframe with the documents we extracted
df = pd.DataFrame(docs)
df.head()

,article_title,section_title,passage_text
0,Terai,History,appointed government officers in the eastern T...
1,WJMX-FM,History,by 1987 before it could be acted upon. In earl...
2,WJMX-FM,History,"""103X"", the CHR format (with more classic hits..."
3,Unified communications,History,"from proprietary vendors. Since March 2008, th..."
4,Tourism in Tunisia,History,to Garrett Nagle in his book Advanced Geograph...


# Initialize Pinecone Index

The Pinecone index stores vector representations of our historical passages which we can retrieve later using another vector (query vector). To build our vector index, we must first establish a connection with Pinecone. For this, we need an API from Pinecone. You can get one for free from [here](https://app.pinecone.io/), and after that, we initialize the connection as follows:

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [90]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

Now we create a new index. We will name it "abstractive-question-answering" — you can name it anything we want. We specify the metric type as "cosine" and dimension as 768 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 768-dimension vectors.

In [89]:
index_name = 'abstractive-question-answering' #give your index a meaningful name

In [105]:
import os
import time
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

PINECONE_API_KEY= os.getenv('PINECONE_API_KEY')

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # create the index if it does not exist
    pc.create_index(
        name=index_name,
        dimension=768,
        metric='cosine',
        spec=spec,
    )
    
index = pc.Index(index_name)
#initialize the index, and insure the stats are all zeros
stats = index.describe_index_stats()
print(stats)


{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '150',
                                    'content-type': 'application/json',
                                    'date': 'Mon, 16 Feb 2026 09:05:37 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '1',
                                    'x-pinecone-request-latency-ms': '1',
                                    'x-pinecone-response-duration-ms': '3'}},
 'dimension': 768,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'storageFullness': 0.0,
 'total_vector_count': 0,
 'vector_type': 'dense'}


# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all historical passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will create embeddings such that the questions and passages that hold the answers to our queries are close to one another in the vector space. We will use a SentenceTransformer model based on Microsoft's MPNet as our retriever. This model performs quite well for comparing the similarity between queries and documents. We can use Cosine Similarity to compute the similarity between query and context vectors generated by this model (Pinecone automatically does this for us).

In [118]:
import torch
print(torch.cuda.is_available())
print(torch.__version__)

True
2.5.1


In [113]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
# retriever = 'flax-sentence-embeddings/all_datasets_v3_mpnet-base' #load the retriever model from HuggingFace. Use the flax-sentence-embeddings/all_datasets_v3_mpnet-base model
retriever="sentence-transformers/all-mpnet-base-v2"
retriever

'sentence-transformers/all-mpnet-base-v2'

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, section title, passage text, etc.

In [119]:
device

'cuda'

In [121]:

# we will use batches of 64
batch_size = 64
model = SentenceTransformer(model_name_or_path=retriever, device=device)

#You will create embedding for the passage_text variable and be use to include the meta data in each batch
for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    # extract batch
    batch = df.iloc[i:i+batch_size, :]
    # generate embeddings for batch
    texts=batch['passage_text'].astype(str).tolist()
    emb=model.encode(texts, batch_size=batch_size)
    emb=emb.tolist()
    titles=batch['article_title'].where(batch['article_title'].notna(),None).tolist()
    # meta=[{'article_title':t, 'passage_text':p} for t,p in zip('article_title', 'passage_text')]
    meta = [{"article_title": t, "passage_text": p} for t, p in zip(batch["article_title"].tolist(), batch["passage_text"].tolist())]
    ids=batch.index.astype(str).tolist()
    
    to_upsert = list(zip(ids,emb, meta))
    test = [to_upsert[0]]
    index.upsert(vectors=test)
    _ = index.upsert(vectors=to_upsert)
# check that we have all vectors in index
index.describe_index_stats()

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


  0%|          | 0/782 [00:00<?, ?it/s]

{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '189',
                                    'content-type': 'application/json',
                                    'date': 'Mon, 16 Feb 2026 12:19:48 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '46',
                                    'x-pinecone-request-latency-ms': '45',
                                    'x-pinecone-response-duration-ms': '47'}},
 'dimension': 768,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'__default__': {'vector_count': 50007}},
 'storageFullness': 0.0,
 'total_vector_count': 50007,
 'vector_type': 'dense'}

# Initialize Generator

We will use ELI5 BART for the generator which is a Sequence-To-Sequence model trained using the ‘Explain Like I’m 5’ (ELI5) dataset. Sequence-To-Sequence models can take a text sequence as input and produce a different text sequence as output.

The input to the ELI5 BART model is a single string which is a concatenation of the query and the relevant documents providing the context for the answer. The documents are separated by a special token &lt;P>, so the input string will look as follows:

>question: What is a sonic boom? context: &lt;P> A sonic boom is a sound associated with shock waves created when an object travels through the air faster than the speed of sound. &lt;P> Sonic booms generate enormous amounts of sound energy, sounding similar to an explosion or a thunderclap to the human ear. &lt;P> Sonic booms due to large supersonic aircraft can be particularly loud and startling, tend to awaken people, and may cause minor damage to some structures. This led to prohibition of routine supersonic flight overland.

More detail on how the ELI5 dataset was built is available [here](https://arxiv.org/abs/1907.09190) and how ELI5 BART model was trained is available [here](https://yjernite.github.io/lfqa.html).

Let's initialize the BART model using transformers.

In [122]:
from transformers import BartTokenizer, BartForConditionalGeneration

# load bart tokenizer and model from huggingface
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa').to(device)

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

c:\ProgramData\miniforge3\envs\llm\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\felip\.cache\huggingface\hub\models--vblagoje--bart_lfqa. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/512 [00:00<?, ?it/s]

All the components of our abstract QA system are complete and ready to be queried. But first, let's write some helper functions to retrieve context passages from Pinecone index and to format the query in the way the generator expects the input.

In [127]:
def query_pinecone(query, top_k):
    # generate embeddings for the query
    xq = model.encode(query).tolist()
    # search pinecone index for context passage with the answer
    xc = index.query(vector=xq, top_k=top_k, include_metadata=True)
    return xc

In [132]:
def format_query(query, context):
    # extract passage_text from Pinecone search result and add the <P> tag
    context = [f"<P> {m['metadata']['passage_text']}" for m in context]
    # concatinate all context passages
    context = " ".join(context) 
    # contcatinate the query and context passages
    query = f"{query}{context}"
    return query

Let's test the helper functions. We will query the Pinecone index function we created earlier with the `query_pinecone` to get context passages and pass them to the `format_query` function.

In [133]:
query = "when was the first electric power system built?"
result = query_pinecone(query, top_k=1)
result

QueryResponse(matches=[{'id': '2584',
 'metadata': {'article_title': 'Electric power system',
              'passage_text': 'These networks were effectively dedicated to '
                              'providing electric lighting. During this time '
                              'the rivalry between Thomas Edison and George '
                              "Westinghouse's companies had grown into a "
                              'propaganda campaign over which form of '
                              'transmission (direct or alternating current) '
                              'was superior, a series of events known as the '
                              '"War of Currents". In 1891, Westinghouse '
                              'installed the first major power system that was '
                              'designed to drive a 100 horsepower (75 kW) '
                              'synchronous electric motor, not just provide '
                              'electric lighting, at Tellu

In [134]:
from pprint import pprint

In [135]:
# format the query in the form generator expects the input
query = format_query(query, result["matches"])
pprint(query)

('when was the first electric power system built?<P> These networks were '
 'effectively dedicated to providing electric lighting. During this time the '
 "rivalry between Thomas Edison and George Westinghouse's companies had grown "
 'into a propaganda campaign over which form of transmission (direct or '
 'alternating current) was superior, a series of events known as the "War of '
 'Currents". In 1891, Westinghouse installed the first major power system that '
 'was designed to drive a 100 horsepower (75 kW) synchronous electric motor, '
 'not just provide electric lighting, at Telluride, Colorado. On the other '
 'side of the Atlantic, Mikhail Dolivo-Dobrovolsky of AEG and Charles Eugene '
 'Lancelot Brown of Maschinenfabrik Oerlikon, built the very first '
 'long-distance (175 km, a distance never tried before) high-voltage (15 kV, '
 'then a record) three-phase transmission line from Lauffen')


The output looks great. Now let's write a function to generate answers.

In [136]:
def generate_answer(query):
    # tokenize the query to get input_ids
    inputs = tokenizer([query], max_length=1024, return_tensors="pt").to(device)
    # use generator to predict output ids
    ids = generator.generate(inputs["input_ids"], num_beams=2, min_length=20, max_length=40)
    # use tokenizer to decode the output ids
    answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return pprint(answer)

In [137]:
generate_answer(query)

('The first electric power system was built in the United States in 1891 by '
 'Thomas Edison and George Westinghouse. They built a transmission line from '
 'Los Angeles to Telluride, Colorado. The')


As we can see, the generator used the provided context to answer our question. Let's run some more queries.

In [138]:
query = "How was the first wireless message sent?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('The first wireless message was sent by a kite in 1866. The kite was attached '
 'to a wire and the kite was attached to a wire. The kite was then attached to')


To confirm that this answer is correct, we can check the contexts used to generate the answer.

In [139]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

transmitted short distances through the air. However most of these systems worked not by radio waves but by electrostatic induction or electromagnetic induction, which had too short a range to be practical. In 1866 Mahlon Loomis claimed to have transmitted an electrical signal through the atmosphere between two 600 foot wires held aloft by kites on mountaintops 14 miles apart. Thomas Edison had come close to discovering radio in 1875; he had generated and detected radio waves which he called "etheric currents" experimenting with high-voltage spark circuits, but due to lack of time did not pursue the matter. David Edward Hughes in 1879 had also stumbled on radio wave transmission which he received with his carbon microphone detector, however he
---
by the U.S. Federal Communications Commission that released the ISM band for unlicensed use. In 1991 NCR Corporation/AT&T (now Nokia Labs and LSI Corporation) invented a precursor to 802.11 in Nieuwegein, the Netherlands. The inventors initia

In this case, the answer looks correct. If we ask a question and no relevant contexts are retrieved, the generator will typically return nonsensical or false answers, like with this question about COVID-19:

In [140]:
query = "where did COVID-19 originate?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('COVID-19 is a zoonotic virus, meaning that it is a virus that can only '
 'infect humans. It is not a virus that can only infect humans. It can only '
 'infect humans')


In [141]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

jumped from other animals to humans, but there is increasing evidence from DNA and RNA sequencing, that measles, smallpox, influenza, HIV, and diphtheria came to humans this way. Various forms of the common cold and tuberculosis also are adaptations of strains originating in other species. Zoonoses are of interest because they are often previously unrecognized diseases or have increased virulence in populations lacking immunity. The West Nile virus appeared in the United States in 1999 in the New York City area, and moved through the country in the summer of 2002, causing much distress. Bubonic plague is a zoonotic disease, as are salmonellosis, Rocky Mountain spotted fever, and Lyme disease. A major factor contributing to the appearance of new zoonotic
---
2002, causing much distress. Bubonic plague is a zoonotic disease, as are salmonellosis, Rocky Mountain spotted fever, and Lyme disease. A major factor contributing to the appearance of new zoonotic pathogens in human populations is

Let’s finish with a final few questions.

In [142]:
query = "what was the war of currents?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('The War of Currents was the rivalry between Thomas Edison and George '
 "Westinghouse's companies over which form of transmission (direct or "
 'alternating current) was superior. The two companies had been competing for')


In [145]:
query = "who was the first person on the moon?"
context = query_pinecone(query, top_k=10)
query = format_query(query, context["matches"])
generate_answer(query)

('The first person to walk on the moon was Neil Armstrong, who walked on the '
 'moon in 1969.')


In [144]:
query = "what was NASAs most expensive project?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('The Space Shuttle was the most expensive project in the history of NASA. It '
 'cost about $1.5 billion to build.')


As we can see, the model can generate some decent answers.

#### Add a few more questions

In [146]:
query = "How did the man got in the moon?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('The man landed on the moon with a parachute. He landed on the moon with a '
 'parachute. He landed on the moon with a parachute. He landed on the moon '
 'with a parachute. He landed')


In [147]:
query = "How many protons has a carbon atom?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('There is no such thing as a "proton" in the sense that it is a particle. It '
 'is a particle that is made up of two quarks and two gluons. The')


In [148]:
query = "who was Thomas Edison?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('Thomas Edison was born in New York City in 1851. He was a chemist, inventor, '
 'and inventor. He was the inventor of the first lightbulb. He was also the '
 'inventor of')
